In [ ]:
import sys
from pathlib import Path

# Chemin du projet 
ROOT = Path().resolve().parent
SRC = ROOT / "src"
sys.path.insert(0, str(SRC))

from td8_discours import read_discours_tsv, split_into_sentences, parse_date_safe

csv_path = ROOT / "data" / "discours_US.csv"

df = read_discours_tsv(csv_path)
df.head()


ModuleNotFoundError: No module named 'td8_discours'

In [ ]:
import sys
from pathlib import Path

ROOT = Path().resolve().parent
SRC = ROOT / "src"

sys.path.insert(0, str(SRC))

from td8_discours import read_discours_tsv, parse_date_safe, split_into_sentences
from Corpus import Corpus
from Document import Document
from SearchEngine import SearchEngine

# 1) Charger le CSV (chemin depuis la racine)
csv_path = ROOT / "data" / "discours_US.csv"
df = read_discours_tsv(csv_path)

# 2) Construire le corpus
Corpus.reset_singleton()
corpus = Corpus("Corpus TD8 - Discours US")

for _, row in df.iterrows():
    speaker = str(row.get("speaker", "Unknown")).strip() or "Unknown"
    title = str(row.get("descr", "Discours")).strip() or "Discours"
    date = parse_date_safe(row.get("date", None))
    url = str(row.get("link", "")).strip()
    text = row.get("text", "")

    for sent in split_into_sentences(text):
        corpus.add_document(
            Document(
                titre=title,
                auteur=speaker,
                date=date,
                url=url,
                texte=sent
            )
        )

# 3) Construire le moteur
se = SearchEngine(corpus)

print("SearchEngine prêt :", corpus)


✅ Séparateur forcé : '\t' (TSV) + lecture robuste (anti-skip)
✅ SearchEngine prêt : Corpus(nom=Corpus TD8 - Discours US, nb_documents=32942, nb_auteurs=2)


In [2]:
import ipywidgets as widgets
from IPython.display import display

query = widgets.Text(
    value="climate change",
    description="Query:",
    layout=widgets.Layout(width="70%")
)

topk = widgets.IntSlider(
    value=10, min=1, max=50, step=1,
    description="Top K:"
)

btn = widgets.Button(description="Search", button_style="success")
out = widgets.Output()

def on_click(_):
    with out:
        out.clear_output()
        res = se.search(query.value, top_k=int(topk.value), show_progress=False)
        display(res)

btn.on_click(on_click)

display(widgets.VBox([widgets.HBox([query, btn]), topk, out]))
